# Install requirements

In [ ]:
%pip install -r /lakehouse/default/Files/SQLMesh.Code/requirements.txt
import sys
sys.exit(0)

# Configuration
Ensure the key vault secret names match the ones created in the specified key vault.

In [ ]:
KEY_VAULT_NAME="kv-ducklake-dev"

FABRIC_WORKSPACE_NAME="WS_DuckLake"
FABRIC_LAKEHOUSE_NAME="LH_DuckLake"
FABRIC_DUCKLAKE_DATA_PATH="DuckLake.Files"
FABRIC_SQLMESH_CODE_PATH="SQLMesh.Code"

FABRIC_LAKEHOUSE_TABLES_BASE_PATH=f"abfss://{FABRIC_WORKSPACE_NAME}@onelake.dfs.fabric.microsoft.com/{FABRIC_LAKEHOUSE_NAME}.Lakehouse/Tables"

# Mapping to the names of the key vault secret names
ENV_VARS = {
    "AZURE_CLIENT_ID": "DuckLakeClientID",
    "AZURE_CLIENT_SECRET": "DuckLakeClientSecret",
    "AZURE_TENANT_ID": "DuckLakeTenantID",
    "PG__HOST": "DuckLakePGHost",
    "PG__PORT": "DuckLakePGPort",
    "PG__DATABASE": "DuckLakePGDatabase",
    "PG__USER": "DuckLakePGUser",
    "PG__PASSWORD": "DuckLakePGPassword",
}


In [ ]:
import os


for env_var_name, secret_name in ENV_VARS.items():
    os.environ[env_var_name] = notebookutils.credentials.getSecret(KEY_VAULT_NAME, secret_name)

os.environ["FABRIC_WORKSPACE_NAME"] = FABRIC_WORKSPACE_NAME
os.environ["FABRIC_LAKEHOUSE_NAME"] = FABRIC_LAKEHOUSE_NAME
os.environ["FABRIC_DUCKLAKE_DATA_PATH"] = FABRIC_DUCKLAKE_DATA_PATH
os.environ["FABRIC_SQLMESH_CODE_PATH"] = FABRIC_SQLMESH_CODE_PATH

In [ ]:
!sqlmesh -p /lakehouse/default/Files/{FABRIC_SQLMESH_CODE_PATH} plan --auto-apply

In [ ]:
import duckdb
import fsspec

storage_options = {
    "account_name": "onelake",
    "account_host": "onelake.blob.fabric.microsoft.com",
    "anon": False,
}

fs = fsspec.filesystem("abfs", **storage_options)

con = duckdb.connect()

con.register_filesystem(fs)

host = os.environ.get("PG__HOST")
database = os.environ.get("PG__DATABASE")
user = os.environ.get("PG__USER")
password = os.environ.get("PG__PASSWORD")

con.execute(f"""
    INSTALL ducklake;
    ATTACH 'ducklake:postgres:
        host={host}
        dbname={database}
        user={user}
        password={password}'
    AS ducklake
"""
)


In [ ]:
from deltalake import write_deltalake

tables = con.sql("""
    USE ducklake; 
    SELECT
        table_catalog,
        table_schema,
        table_name
    FROM INFORMATION_SCHEMA.TABLES 
    WHERE
        table_schema IN ('silver','gold')
    """).pl().to_dicts()


for table in tables:
    
    catalog = table.get("table_catalog")
    schema = table.get("table_schema")
    name = table.get("table_name")

    print(f"Syncing ducklake table: {catalog}.{schema}.{name} to delta table: {schema}.{name}")

    data = con.execute(f"SELECT * FROM {catalog}.{schema}.{name}").arrow()

    write_deltalake(
        table_or_uri=f"{FABRIC_LAKEHOUSE_TABLES_BASE_PATH}/{schema}/{name}",
        data=data,
        mode="overwrite",
        schema_mode="overwrite",
    )
    